# URLs
|section|URL|
|-|-|
|Server Features|http://`<server:port>`/?X-Plex-Token=`<token>`|
|Movies|http://`<server:port>`/library/sections/`<id>`/all?X-Plex-Token=`<token>`|
|TV Episodes|http://`<server:port>`/library/sections/`<id>`/allLeaves?X-Plex-Token=`<token>`|

In [9]:
import pandas as pd
from os import getenv
from dotenv import load_dotenv

# load from .env file
load_dotenv()

site_url = 'http://nas.lan:32400'
token = getenv('PLEX_TOKEN')
movie_index = 3
tv_index = 6

## Grab Movie Data

In [10]:
movies_collection = pd.read_xml(
    path_or_buffer = f'{site_url}/library/sections/{movie_index}/all?X-Plex-Token={token}',
    parser = 'lxml',
    stylesheet='movie_transform.xsl',
    parse_dates=['release']
)

Seperate folders from pathnames, then remove full_path

In [11]:
movies_collection[['folder', 'file']] = movies_collection['full_path'].str.extract('(.*)\/([^\/]*)$', expand=True)
movies_collection = movies_collection.drop(columns=['full_path'])

## Grab TV data

In [12]:
tv_collection = pd.read_xml(
    path_or_buffer = f'{site_url}/library/sections/{tv_index}/allLeaves?X-Plex-Token={token}',
    parser = 'lxml',
    stylesheet='tv_transform.xsl',
    parse_dates=['release']
)

Seperate folders from pathnames, then remove full_path

In [13]:
tv_collection[['folder', 'file']] = tv_collection['full_path'].str.extract('(.*)\/([^\/]*)$', expand=True)
tv_collection = tv_collection.drop(columns=['full_path'])

### Output data

In [14]:

with pd.ExcelWriter("data.xlsx") as datafile:
    movies_collection.to_excel(datafile, sheet_name='Movies', index=None)
    tv_collection.to_excel(datafile, sheet_name='TV Shows', index=None)

movies_collection.to_csv('movie_data.csv', index=None)
tv_collection.to_csv('tv_data.csv', index=None)